In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch

In [2]:
DATASET_DIR = "C:/Users/db/Desktop/기업-수강ncf/Dataset"

In [3]:
data_against = pd.read_csv(DATASET_DIR+"/학습용 입력 데이터 (역학기 36,375개).csv")[['BZC_CD', 'GRUP_CD']]
data_normal = pd.read_csv(DATASET_DIR+"/학습용 입력 데이터 (정상학기 4,337,198개).csv")[['BZC_CD', 'GRUP_CD']]
data_normal =  data_normal.dropna()
data_against = data_against.dropna()

C:\Users\db\.conda\envs\3.7_imagetuto\lib\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# data_normal = pd.read_csv('/content/drive/MyDrive/ncf/학습용 입력 데이터 (역학기 36,375개).csv')

In [5]:
data_normal

,BZC_CD,GRUP_CD
565,41229.0,356
566,41229.0,6808
567,41229.0,5575
568,41229.0,298
569,41229.0,354
...,...,...
4330341,72121.0,4153
4330342,72121.0,4445
4330343,72121.0,4128
4330344,72121.0,4446


In [6]:
rating_list = np.concatenate([data_normal.values, data_against.values])
rating_list

array([[41229.,   356.],
       [41229.,  6808.],
       [41229.,  5575.],
       ...,
       [21210.,  5098.],
       [21210.,  5110.],
       [21210.,  6194.]])

In [7]:
rating_list = np.concatenate([data_normal.values, data_against.values])
n_rating_list = []
for a in rating_list:
     st, cd = a
     n_rating_list.append([st, cd])
rating_list = np.array(n_rating_list)

In [8]:
# 회사가 user 과목이 item으로 저장되어 있음
rating_list

array([[41229.,   356.],
       [41229.,  6808.],
       [41229.,  5575.],
       ...,
       [21210.,  5098.],
       [21210.,  5110.],
       [21210.,  6194.]])

In [9]:
# 모든 유저와 모든 item을 담음
unq_user, unq_item = [np.unique(rating_list[:, n]) for n in range(2)]

In [10]:
# user와 item의 개수 넣음
num_user, num_item = len(unq_user), len(unq_item)

In [11]:
num_user

922

In [12]:
# 각인덱스를 매핑해놓음
user2idx, item2idx = {}, {}
for i, j in enumerate(unq_user):
     user2idx[j] = i
for i, j in enumerate(unq_item):
     item2idx[j] = i

In [13]:
item2idx

{1.0: 0,
 2.0: 1,
 3.0: 2,
 4.0: 3,
 5.0: 4,
 6.0: 5,
 7.0: 6,
 12.0: 7,
 15.0: 8,
 19.0: 9,
 21.0: 10,
 23.0: 11,
 24.0: 12,
 25.0: 13,
 26.0: 14,
 27.0: 15,
 28.0: 16,
 29.0: 17,
 30.0: 18,
 32.0: 19,
 33.0: 20,
 34.0: 21,
 39.0: 22,
 40.0: 23,
 42.0: 24,
 45.0: 25,
 47.0: 26,
 48.0: 27,
 49.0: 28,
 50.0: 29,
 51.0: 30,
 52.0: 31,
 53.0: 32,
 55.0: 33,
 56.0: 34,
 57.0: 35,
 58.0: 36,
 59.0: 37,
 60.0: 38,
 61.0: 39,
 62.0: 40,
 63.0: 41,
 64.0: 42,
 65.0: 43,
 66.0: 44,
 67.0: 45,
 68.0: 46,
 69.0: 47,
 70.0: 48,
 71.0: 49,
 72.0: 50,
 73.0: 51,
 74.0: 52,
 75.0: 53,
 76.0: 54,
 77.0: 55,
 78.0: 56,
 79.0: 57,
 80.0: 58,
 81.0: 59,
 82.0: 60,
 83.0: 61,
 84.0: 62,
 85.0: 63,
 86.0: 64,
 88.0: 65,
 89.0: 66,
 90.0: 67,
 91.0: 68,
 93.0: 69,
 94.0: 70,
 95.0: 71,
 96.0: 72,
 97.0: 73,
 98.0: 74,
 99.0: 75,
 100.0: 76,
 102.0: 77,
 103.0: 78,
 104.0: 79,
 105.0: 80,
 106.0: 81,
 108.0: 82,
 109.0: 83,
 110.0: 84,
 111.0: 85,
 112.0: 86,
 113.0: 87,
 115.0: 88,
 117.0: 89,
 118.0: 90,
 

In [14]:
# 위에서 매핑한 인덱스로 각아이템과 user를 바꿔줌
rating_list = [[user2idx[li[0]], item2idx[li[1]]] for li in rating_list]

In [15]:
rating_list

[[451, 280],
 [451, 4791],
 [451, 3912],
 [451, 231],
 [451, 278],
 [451, 237],
 [451, 283],
 [451, 290],
 [451, 234],
 [451, 281],
 [451, 2938],
 [451, 3638],
 [451, 3638],
 [451, 740],
 [451, 236],
 [451, 242],
 [451, 279],
 [451, 4791],
 [451, 4792],
 [451, 276],
 [451, 4674],
 [451, 3968],
 [518, 3183],
 [518, 3287],
 [518, 3187],
 [518, 3189],
 [518, 3185],
 [518, 3188],
 [518, 4335],
 [518, 3287],
 [518, 5251],
 [518, 3190],
 [518, 3221],
 [518, 3287],
 [518, 3196],
 [518, 3194],
 [518, 3195],
 [518, 3192],
 [518, 3191],
 [518, 3182],
 [518, 3178],
 [518, 3176],
 [518, 3177],
 [518, 3198],
 [518, 3199],
 [518, 3212],
 [518, 3202],
 [518, 3208],
 [518, 3286],
 [518, 3205],
 [518, 3206],
 [518, 3207],
 [518, 3203],
 [518, 3204],
 [518, 3200],
 [518, 3214],
 [518, 3211],
 [518, 3210],
 [518, 3209],
 [518, 3201],
 [518, 3213],
 [518, 3215],
 [518, 3217],
 [518, 3238],
 [518, 3286],
 [518, 3179],
 [518, 3218],
 [518, 3219],
 [518, 3216],
 [518, 3638],
 [518, 3230],
 [518, 3222],
 [518

In [16]:
mat = sp.dok_matrix((num_user, num_item), dtype=np.float32)

In [17]:
for line in rating_list:
   u, i = line
   mat[u, i] = 1

In [18]:
# 0번째 사용자가 4,139,25,,, 등등에 1.0을 주었다는 뜻뜻
print(mat)

  (451, 280)	1.0
  (451, 4791)	1.0
  (451, 3912)	1.0
  (451, 231)	1.0
  (451, 278)	1.0
  (451, 237)	1.0
  (451, 283)	1.0
  (451, 290)	1.0
  (451, 234)	1.0
  (451, 281)	1.0
  (451, 2938)	1.0
  (451, 3638)	1.0
  (451, 740)	1.0
  (451, 236)	1.0
  (451, 242)	1.0
  (451, 279)	1.0
  (451, 4792)	1.0
  (451, 276)	1.0
  (451, 4674)	1.0
  (451, 3968)	1.0
  (518, 3183)	1.0
  (518, 3287)	1.0
  (518, 3187)	1.0
  (518, 3189)	1.0
  (518, 3185)	1.0
  :	:
  (387, 342)	1.0
  (387, 343)	1.0
  (387, 344)	1.0
  (387, 341)	1.0
  (387, 353)	1.0
  (387, 337)	1.0
  (387, 350)	1.0
  (387, 349)	1.0
  (387, 348)	1.0
  (387, 359)	1.0
  (387, 355)	1.0
  (387, 4334)	1.0
  (387, 357)	1.0
  (387, 4593)	1.0
  (387, 370)	1.0
  (387, 4582)	1.0
  (387, 368)	1.0
  (387, 356)	1.0
  (829, 5016)	1.0
  (829, 5373)	1.0
  (829, 5013)	1.0
  (829, 5211)	1.0
  (829, 5315)	1.0
  (829, 5015)	1.0
  (829, 5014)	1.0


In [19]:
matd = np.array(mat.todense())

In [20]:
matd

array([[1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [21]:
user_id = 50
user_ratings = matd[user_id]
items_rated_by_user = []
for i, rating in enumerate(user_ratings):
    if rating != 0:
        items_rated_by_user.append((i))
print(items_rated_by_user)

[298, 851, 951, 952, 953, 956, 957, 958, 961, 962, 963, 964, 965, 966, 968, 970, 976, 977, 982, 983, 985, 991, 992, 993, 994, 995, 996, 1137, 1138, 1140, 1145, 1151, 1203, 1213, 1217, 1218, 1219, 1220, 1223, 1228, 1229, 1231, 1235, 1245, 1259, 1273, 1275, 1276, 1278, 1279, 1280, 1282, 1283, 1284, 1288, 1427, 1637, 2439, 2469, 2665, 2713, 2780, 2935, 2936, 2938, 2939, 2940, 2941, 2942, 2943, 2944, 2945, 2946, 2948, 2949, 2951, 2952, 2953, 2954, 2955, 2956, 2958, 2959, 2961, 2963, 2964, 2966, 2970, 2975, 3068, 3638, 3642, 3745, 3970, 3971, 3972, 4238, 4335, 4790, 4791, 4792, 4799, 4868, 4877, 4922, 5209, 5403]


In [22]:
num_item

5463

In [23]:
# negative인 값을 train에서 뽑음 99가지를 뽑음
num_negative = 99
test_data = []
for v in range(len(user2idx.keys())):
    temp = []
    for n in range(num_negative):
       idx = np.random.randint(0, len(item2idx.keys()))
       while matd[v][idx] != 0 or idx not in temp:
              idx = np.random.randint(0, len(item2idx.keys()))
              temp.append(idx)
       test_data.append([v, idx])

In [24]:
len(test_data)

91278

In [25]:
test_data

[[0, 2923],
 [0, 4746],
 [0, 5143],
 [0, 2817],
 [0, 3540],
 [0, 3692],
 [0, 4041],
 [0, 5270],
 [0, 2224],
 [0, 4983],
 [0, 3165],
 [0, 4578],
 [0, 1379],
 [0, 3352],
 [0, 3325],
 [0, 2097],
 [0, 644],
 [0, 2220],
 [0, 1727],
 [0, 4666],
 [0, 215],
 [0, 1555],
 [0, 2800],
 [0, 509],
 [0, 3203],
 [0, 1081],
 [0, 3838],
 [0, 2785],
 [0, 3954],
 [0, 2309],
 [0, 716],
 [0, 4361],
 [0, 2021],
 [0, 1193],
 [0, 1257],
 [0, 3644],
 [0, 4081],
 [0, 1918],
 [0, 3315],
 [0, 260],
 [0, 465],
 [0, 3334],
 [0, 2963],
 [0, 3436],
 [0, 3982],
 [0, 1891],
 [0, 394],
 [0, 560],
 [0, 4139],
 [0, 4976],
 [0, 2119],
 [0, 4997],
 [0, 880],
 [0, 3602],
 [0, 3961],
 [0, 3171],
 [0, 1312],
 [0, 1134],
 [0, 1768],
 [0, 3860],
 [0, 4727],
 [0, 107],
 [0, 1242],
 [0, 1010],
 [0, 3315],
 [0, 3483],
 [0, 1670],
 [0, 3251],
 [0, 1508],
 [0, 867],
 [0, 5235],
 [0, 5380],
 [0, 2681],
 [0, 4512],
 [0, 1858],
 [0, 4649],
 [0, 3743],
 [0, 2113],
 [0, 2760],
 [0, 1390],
 [0, 5202],
 [0, 2393],
 [0, 4891],
 [0, 284],
 [0,

In [26]:
# rating_list : num과 item이 들어가있음
# test_data : 안들은 negative 샘플이 들어있음
# num_user : user의 개수
# num_item : item의 개수
# mat: num_user * num_item의 개수

In [27]:
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np

In [28]:
class GMF(nn.Module):
    def __init__(self, user_num, item_num, hidden, dropout):
        super(GMF, self).__init__()
        self.dropout = dropout

        self.embed_user = nn.Embedding(user_num, hidden)
        self.embed_item = nn.Embedding(item_num, hidden)

        self.predict_layer = nn.Linear(hidden, 1)
        self._init_weight_()

    def _init_weight_(self):
        nn.init.normal_(self.embed_user.weight, std=0.001)
        nn.init.normal_(self.embed_item.weight, std=0.001)
        nn.init.kaiming_uniform_(self.predict_layer.weight, a=1, nonlinearity='sigmoid')

        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, user, item):
        embed_user = self.embed_user(user)
        embed_item = self.embed_item(item)
        output = embed_user * embed_item
        pred = self.predict_layer(output)
        return pred.view(-1)

In [29]:
class NCFData(torch.utils.data.Dataset):
    def __init__(self, features, num_item, train_mat=None, num_ng=0, is_training=None):
        super(NCFData, self).__init__()
        """ Note that the labels are only useful when training, we thus
			add them in the ng_sample() function.
		"""
        # self.features_ps = [[0, 121], [0, 199], [1, 456],...]
        self.features_ps = features
        self.num_item = num_item
        self.train_mat = train_mat
        self.num_ng = num_ng
        self.is_training = is_training
        self.labels = [0] * len(features)

    def set_ng_sample(self):
        assert self.is_training, "no need to sampling when testing"

        # negative sample 더하기
        self.features_ng = []
        for x in self.features_ps:
            # user
            u = x[0]
            for _ in range(self.num_ng):
                j = np.random.randint(self.num_item)
                # train set에 있는 경우 다시 뽑기
                while (u, j) in self.train_mat:
                    j = np.random.randint(self.num_item)
                self.features_ng.append([u, j])

        labels_ps = [1] * len(self.features_ps)
        labels_ng = [0] * len(self.features_ng)

        self.features_fill = self.features_ps + self.features_ng
        self.labels_fill = labels_ps + labels_ng

    def __len__(self):
        return (self.num_ng + 1) * len(self.labels)

    def __getitem__(self, idx):
        features = self.features_fill if self.is_training else self.features_ps
        labels = self.labels_fill if self.is_training else self.labels

        user = features[idx][0]
        item = features[idx][1]
        label = labels[idx]
        return user, item, label

In [30]:
def prepare_data(train_data, test_data, item_num, train_mat):
    train_dataset = NCFData(train_data, item_num, train_mat, 4, True)
    test_dataset = NCFData(test_data, item_num, train_mat, 0, False)

    train_loader = data.DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=0)
    test_loader = data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=0)

    return train_loader, test_loader

In [31]:
len(train_loader)

NameError: name 'train_loader' is not defined

In [ ]:
def hit(gt_item, pred_items):
    if gt_item in pred_items:
        return 1
    return 0

In [ ]:
def ndcg(gt_item, pred_items):
    if gt_item in pred_items:
        index = pred_items.index(gt_item)
        return np.reciprocal(np.log2(index + 2))
    return 0


In [ ]:
def metrics(net, test_loader, topk):
    HR, NDCG = [], []

    for user, item, _ in test_loader:
        user = user.cuda()
        item = item.cuda()

        pred = net(user, item)
        if len(pred) < topk:
            continue
        _, indices = torch.topk(pred, topk)
        recommends = torch.take(item, indices).cpu().numpy().tolist()

        gt_item = item[0].item()
        HR.append(hit(gt_item, recommends))
        NDCG.append(ndcg(gt_item, recommends))
    return np.mean(HR), np.mean(NDCG)


In [ ]:
train_data, test_data, num_user, num_item, train_mat = rating_list, test_data, num_user, num_item, mat

In [ ]:
train_loader, test_loader = prepare_data(train_data, test_data, num_item, train_mat)

In [ ]:
# data, target = next(iter(test_loader))

In [ ]:
epochs = 300
hidden = 64
dropout = 0.2
learning_rate = 0.005
topk = 50

In [ ]:
net = GMF(num_user, num_item, hidden, dropout)
device = 'cuda:0'
net.to(device)

In [ ]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
for ep in range(epochs):
    avg_cost = 0
    net.train()
    train_loader.dataset.set_ng_sample()
    
    for user, item, label in train_loader:
        user = user.cuda()
        item = item.cuda()
        label = label.float().cuda()

        net.zero_grad()
        pred = net(user, item)
        loss = loss_function(pred, label)
        loss.backward()
        optimizer.step()
        avg_cost += loss / len(train_loader)
    print('Epoch:', '%04d' % (epochs + 1), 'cost =', '{:.9f}'.format(avg_cost)) 

In [ ]:
#  torch.save(net,'./net.pth')

In [ ]:
net = torch.load('/content/drive/MyDrive/ncf_final/net.pth')

In [ ]:
set1 = set(items_rated_by_user)
set2 = set(ids)
intersection = set1.intersection(set2)
result = list(intersection)

In [ ]:
ids = [x[0] for x in sorted(aa, reverse=True, key=lambda x: x[1])[:k]]
print(ids)

In [ ]:
k = 10
device = 'cuda:0'
user_id = 50
user = torch.LongTensor([user_id]).to(device)
num_item = 5463


aa = []
for i in range(num_item):
    item = torch.LongTensor([i]).to(device)
    pred = net(user, item).item()
    aa.append([i, pred])
print(len(aa))
print(aa[:k])
print(sorted(aa, reverse=True, key=lambda x: x[1])[:k])


In [ ]:
test_list

In [ ]:
HR, NDCG = metrics(net, test_loader, 10)
print('HR : %f, NDCG : %f' % (np.mean(HR), np.mean(NDCG)))

In [ ]:
HR, NDCG = metrics(net, test_loader, 50)
print('HR : %f, NDCG : %f' % (np.mean(HR), np.mean(NDCG)))

In [ ]:
# HR, NDCG = metrics(net,  test_loader, )
# print('HR : %f, NDCG : %f' % (np.mean(HR), np.mean(NDCG)))

In [ ]:
# hitrate
# NDCG

In [ ]:
rating_list

In [ ]:
unique_values = list(set([item[0] for item in rating_list]))

print(unique_values)  # [565, 697]

In [ ]:
# 총회사는 921개

In [ ]:
import numpy as np

k = 10
device = 'cuda:0'
user_id = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
user = torch.LongTensor(np.array(user_id)).to(device)
num_item = 14000

aa = []
for i in range(num_item):
    item = torch.LongTensor([i]).to(device)
    pred = net(user, item).item()
    aa.append([i, pred])

print(aa[:k])
print(sorted(aa, reverse=True, key=lambda x: x[1])[:k])

In [ ]:
items_rated_by_user